# Pulling Live Data for React Databoard  
  
Goals:
* Use FinnHub API to pull data
* Convert API calls into a script
* Run script from dashboard to load data

In [1]:
import finnhub

In [69]:
import os
from dotenv import load_dotenv

load_dotenv()
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")
FMP_API_KEY = os.getenv("FMP_API_KEY")
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

user_client = finnhub.Client(api_key=FINNHUB_API_KEY)

In [77]:
import requests


ticker = "GOOGL"
av_api_url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={FMP_API_KEY}"
av_req = requests.get(fmp_api_url)
av_req

<Response [200]>

In [78]:
av_data = av_req.json()["quarterlyEarnings"]
av_data

[{'fiscalDateEnding': '2025-09-30',
  'reportedDate': '2025-10-28',
  'reportedEPS': '3.0917',
  'estimatedEPS': '2.33',
  'surprise': '0.7617',
  'surprisePercentage': '32.691',
  'reportTime': 'post-market'},
 {'fiscalDateEnding': '2025-06-30',
  'reportedDate': '2025-07-23',
  'reportedEPS': '2.31',
  'estimatedEPS': '2.2',
  'surprise': '0.11',
  'surprisePercentage': '5',
  'reportTime': 'post-market'},
 {'fiscalDateEnding': '2025-03-31',
  'reportedDate': '2025-04-24',
  'reportedEPS': '2.81',
  'estimatedEPS': '2.01',
  'surprise': '0.8',
  'surprisePercentage': '39.801',
  'reportTime': 'post-market'},
 {'fiscalDateEnding': '2024-12-31',
  'reportedDate': '2025-02-04',
  'reportedEPS': '2.15',
  'estimatedEPS': '2.11',
  'surprise': '0.04',
  'surprisePercentage': '1.8957',
  'reportTime': 'post-market'},
 {'fiscalDateEnding': '2024-09-30',
  'reportedDate': '2024-10-28',
  'reportedEPS': '2.12',
  'estimatedEPS': '1.85',
  'surprise': '0.27',
  'surprisePercentage': '14.5946',

In [48]:
x["peTTM"][:5], x["eps"][:5]

([{'period': '2025-09-30', 'v': 23.6486},
  {'period': '2025-06-30', 'v': 18.5112},
  {'period': '2025-03-31', 'v': 17.0128},
  {'period': '2024-12-31', 'v': 23.2079},
  {'period': '2024-09-30', 'v': 21.6553}],
 [{'period': '2025-09-30', 'v': 2.8664},
  {'period': '2025-06-30', 'v': 2.3115},
  {'period': '2025-03-31', 'v': 2.8102},
  {'period': '2024-12-31', 'v': 2.0086},
  {'period': '2024-09-30', 'v': 2.1178}])

In [57]:
class UserClient:
    def __init__(self, user_api_key):
        self.client = finnhub.Client(api_key=user_api_key)

    def get_dashboard_data(self, ticker):
        basic_financial_info = self.client.company_basic_financials(ticker, "all")

        ttm_eps = basic_financial_info["metric"]["epsTTM"]
        recent_eps = basic_financial_info["series"]["quarterly"]["eps"][:5] # comparing prev quarter to 1 year prior
        yoy_quarterly_eps_change = recent_eps[0]["v"] / recent_eps[-1]["v"] - 1

        current_price = self.client.quote(ticker)["c"]

        previous_pre_earnings_ttm_pe = basic_financial_info["series"]["quarterly"]["peTTM"][0]["v"]
        previous_pre_earnings_price = ttm_eps * previous_pre_earnings_ttm_pe
        print("we found eps of ", ttm_eps, " and pe of ", previous_pre_earnings_ttm_pe) 
        print("macrotrends expects a calculated price of ", 10.13 * 24.03)

        dashboard_data = [ttm_eps, yoy_quarterly_eps_change, current_price, previous_pre_earnings_price]
        return dashboard_data
    

u2 = UserClient(user_api_key=FINNHUB_API_KEY)
u2.get_dashboard_data("GOOG")

we found eps of  9.9967  and pe of  23.6486
macrotrends expects a calculated price of  243.42390000000003


[9.9967, 0.35348002644253484, 336.43, 236.40795961999999]

In [47]:
ticker = "GOOG"

x = user_client.company_basic_financials(ticker, "all")["series"]["quarterly"]
relevant_features = [
    "epsTTM", # for E/P. From user_client.company_basic_financials(ticker, "all")["metric"]["epsTTM"]
    "eps", # for YOY quarterly Earnings Change (%). From user_client.company_basic_financials(ticker, "all")["series"]["quarterly"]["eps"]
    "quote", # for Price Change (%) since previous quarter's earnings
    "peTTM", # can be combined with eps to find the previous share price before the earnings calls.
]
list(x.keys())

['assetTurnoverTTM',
 'bookValue',
 'cashRatio',
 'currentRatio',
 'ebitPerShare',
 'eps',
 'ev',
 'evEbitdaTTM',
 'evRevenueTTM',
 'fcfMargin',
 'fcfPerShareTTM',
 'grossMargin',
 'inventoryTurnoverTTM',
 'longtermDebtTotalAsset',
 'longtermDebtTotalCapital',
 'longtermDebtTotalEquity',
 'netDebtToTotalCapital',
 'netDebtToTotalEquity',
 'netMargin',
 'operatingMargin',
 'payoutRatioTTM',
 'pb',
 'peTTM',
 'pfcfTTM',
 'pretaxMargin',
 'psTTM',
 'ptbv',
 'quickRatio',
 'receivablesTurnoverTTM',
 'roaTTM',
 'roeTTM',
 'roicTTM',
 'rotcTTM',
 'salesPerShare',
 'sgaToSale',
 'tangibleBookValue',
 'totalDebtToEquity',
 'totalDebtToTotalAsset',
 'totalDebtToTotalCapital',
 'totalRatio']